Ordering and Delivery to Google Earth Engine (GEE)

Based off of the client's example (https://github.com/planetlabs/planet-client-python/blob/main/examples/orders_create_and_download_multiple_orders.py)

In [1]:
import asyncio
import os

import planet

Authetication

In [2]:
# API Keys stored as an env variable
PLANET_API_KEY = os.getenv('PL_API_KEY')
GEE_API_KEY = os.getenv('GEE_API_KEY')
print(GEE_API_KEY)

None


Define order

In [ ]:
DOWNLOAD_DIR = os.getenv('TEST_DOWNLOAD_DIR', '.')

iowa_aoi = {
    "type":
    "Polygon",
    "coordinates": [[[-91.198465, 42.893071], [-91.121931, 42.893071],
                     [-91.121931, 42.946205], [-91.198465, 42.946205],
                     [-91.198465, 42.893071]]]
}

iowa_images = ['20200925_161029_69_2223', '20200925_161027_48_2223']

Define cloud delivery location

In [ ]:
google_cloud_storage_config = {
        "delivery": {
            "google_cloud_storage": {
                "project": "planet-earthengine-staging",
                "collection": "gee-integration-testing",
                "credentials": GEE_API_KEY
            }
    }
}
delivery_config = planet.order_request.delivery(
    cloud_config=planet.order_request.google_earth_engine(project='planet-earthengine-staging', collection='gee-integration-testing')
)

Build order request

In [ ]:
iowa_order = planet.order_request.build_request(
    'iowa_order',
    [planet.order_request.product(iowa_images, 'analytic', 'PSScene4Band')],
    delivery=planet.order_request.delivery(cloud_config=delivery_config),
    tools=[planet.order_request.clip_tool(iowa_aoi)])

print(iowa_order)

Define create and download order functions

In [ ]:
async def create_and_download(order_detail, directory, client):
    with planet.reporting.StateBar(state='creating') as reporter:
        # create
        order = await client.create_order(order_detail)
        reporter.update(state='created', order_id=order['id'])
        # wait for completion
        await client.wait(order['id'], callback=reporter.update_state)

    order_details = await client.get_order(order_id=order['id'])
    return order_details

Create and download order

In [ ]:
async with planet.Session() as ps:
    client = planet.OrdersClient(ps)
    order_details = await create_and_download(iowa_order, DOWNLOAD_DIR, client)

In [ ]:
print(order_details['id'])
print(order_details['_links']['results'][0])